In [30]:
import numpy as np
from torchvision import transforms
from torch.utils.data import DataLoader, TensorDataset
import os
import torch
from torchvision import transforms
import torchvision.datasets as datasets
import torch.nn as nn
import torch.nn.functional as F
from torchvision.models import resnet50, ResNet50_Weights
from torchvision.models import efficientnet_v2_m, EfficientNet_V2_M_Weights
from sklearn.preprocessing import normalize
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
import pandas as pd
from sklearn.model_selection import train_test_split

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [31]:
class MyDataset(Dataset):
    def __init__(self,dataframe,label):
        self.data = dataframe
        self.label = label
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self,idx):
        return nn.tensor(self.data[idx]),self.label

In [32]:
spectograms_orig = pd.read_csv('spectograms_train.csv')
y_train_orig = pd.read_csv('y_train.csv')


In [33]:
def generate_embeddings(dl_train):
    """
    Transform, resize and normalize the images and then use a pretrained model to extract
    the embeddings.
    """
    embedding_size = 1000  # Dummy variable, replace with the actual embedding size once you pick your model
    batch_size = 16

    # TODO: define a transform to pre-process the images
    train_transforms = EfficientNet_V2_M_Weights.DEFAULT.transforms()  # transforms.Compose([transforms.ToTensor()])

    train_dataset = datasets.ImageFolder(root="dataset/", transform=train_transforms)
    
    print((train_dataset[1][1]))
    # Hint: adjust batch_size and num_workers to your PC configuration, so that you don't run out of memory
    train_loader = dl_train

    # TODO: define a model for extraction of the embeddings (Hint: load a pretrained model,
    #  more info here: https://pytorch.org/vision/stable/models.html)
    
    model = efficientnet_v2_m(weights=EfficientNet_V2_M_Weights.DEFAULT)

    embeddings = np.array([], dtype=np.float32).reshape(0, embedding_size)
    num_images = len(train_dataset)
    # embeddings = np.zeros((num_images, embedding_size))
    # TODO: Use the model to extract the embeddings. Hint: remove the last layers of the
    # model to access the embeddings the model generates.

    # model = torch.nn.Sequential(*(list(model.children())[:-1]))
    model.to(device)
    idx = 0
    with torch.no_grad():
        for batch, _ in train_loader:
            batch = batch.cuda()
            model.eval()
            preds = model.forward(batch).cpu().numpy()
            preds = preds.reshape((int(preds.size / embedding_size), embedding_size))
            embeddings = np.concatenate((embeddings, preds), axis=0)
            idx += batch_size
            print(f"{idx / num_images * 100} %")

    np.save("dataset/embeddings.npy", embeddings)


In [34]:
X_train, X_test, y_train, y_test = train_test_split(spectograms_orig, y_train_orig)
train_dataset = MyDataset(X_train,y_train)
test_dataset = MyDataset(X_test,y_test)

dl_train = DataLoader(train_dataset,batch_size=64,shuffle=True,num_workers=4)
dl_test = DataLoader(test_dataset,batch_size=64,shuffle=True,num_workers=4)

In [35]:
generate_embeddings(dl_train)

0
